In [1]:
import os
os.environ["JAVA_HOME"] = "/mnt/extproj/projekte/textmining/markus_tm_projs/jdk/openlogic-openjdk-11.0.22+7-linux-x64"

MXPLORE_PATH="/mnt/extproj/projekte/textmining/mxplore/"

from collections import Counter

In [2]:
from collections import defaultdict
from pyspark.sql import SparkSession

from pyspark.sql.functions import *
from pyspark.sql.types import *

from pyspark.sql import Window
import glob

In [3]:
if not "spark" in globals() or spark is None:
    print("Creating builder")
    spark = SparkSession.builder.\
            config("spark.executor.memory", "70g").\
            config("spark.driver.memory", "50g").\
            config("spark.memory.offHeap.enabled",True).\
            config("spark.memory.offHeap.size","16g").\
            config("spark.sql.shuffle.partitions", 300).\
            appName('mirexplore').getOrCreate()
    
#spark.conf.set("spark.executor.memory", "70g")
#spark.conf.set("spark.driver.memory", "50g")
#spark.conf.set("spark.memory.offHeap.enabled",True)
#spark.conf.set("spark.memory.offHeap.size","16g")  
#spark.conf.set("spark.sql.shuffle.partitions", 300)

Creating builder


/mnt/extproj/projekte/bartelt/software/miniconda3/envs/mirexplore/lib/python3.6/site-packages/pyspark/context.py:238: FutureWarning: Python 3.6 support is deprecated in Spark 3.2.
  FutureWarning


In [4]:
df_int = spark.read.parquet(MXPLORE_PATH+"/mxresults/mx_mirna_gene_consensus_parquet/")
df_int = df_int.withColumn("doc_id", explode("evidence_documents"))

In [5]:
relDocIDs = set([x["doc_id"] for x in df_int.select("doc_id").collect()])
print(len([x for x in relDocIDs if x.startswith("PMC")]))

55331


In [6]:
def read_date_file(inputpath, relevantDocIDs):
    db_entries = list()
    ignoredConcepts = Counter()


    for infile in glob.glob("{}/*.journal".format(inputpath)):
        with open(infile, "r") as fin:
            
            for iline, line in enumerate(fin):
                aline = line.split("\t")
                aline = [x.strip() for x in aline]

                docID = aline[0]
                
                if not docID in relevantDocIDs:
                    continue
                
                if len(aline) < 2:
                    continue
                
                title = aline[1]

                db_entries.append( (docID, title) )
                
    return db_entries


In [7]:
docAuthors = list(set(read_date_file("/mnt/extproj/projekte/textmining/pubmed_feb24/", relDocIDs)))
print(len(docAuthors))

47280


In [8]:
docAuthorsPMC = list(set(read_date_file("/mnt/extproj/projekte/textmining/pmc_feb24/oa_comm/", relDocIDs)))
print(len(docAuthorsPMC))

55331


In [9]:
df = spark.createDataFrame(docAuthors+docAuthorsPMC, ["doc_id", "title"])

In [10]:
df.show()

+--------+--------------------+
|  doc_id|               title|
+--------+--------------------+
|31805275|Experimental cell...|
|28550686|   Oncology research|
|27296666|Journal of immuno...|
|29848672| Anticancer research|
|36284637| Journal of oncology|
|26222820|Clinics (Sao Paul...|
|28273432|The American jour...|
|23564324|Medical oncology ...|
|37467842|Experimental neur...|
|24998573|Asian Pacific jou...|
|34288057|Journal of cellul...|
|26457240|Current pharmacol...|
|35116871|Translational can...|
|33481955|            PloS one|
|35579934|Annals of the New...|
|36101795|Contrast media & ...|
|33714676|Asian journal of ...|
|26025955|Cardiovascular re...|
|37186186|Environmental sci...|
|30384358|Cellular physiolo...|
+--------+--------------------+
only showing top 20 rows



In [11]:
df.write.parquet(MXPLORE_PATH+"/mxresults/mx_document_title", mode="overwrite")

In [12]:
df.orderBy("doc_id").show()

+--------+--------------------+
|  doc_id|               title|
+--------+--------------------+
|10760272|Proceedings of th...|
|12576545|Proceedings of th...|
|12812784|Developmental bio...|
|15003116|      Genome biology|
|15361871|     Nature genetics|
|15504739|The Journal of bi...|
|15538371|              Nature|
|15634332|    The FEBS journal|
|15648093|Journal of cellul...|
|15738415|Proceedings of th...|
|15766526|                Cell|
|15944709|              Nature|
|16041695|The Journal of pa...|
|16096373|Cell cycle (Georg...|
|16151463|            Leukemia|
|16166262|Proceedings of th...|
|16227573|Molecular and cel...|
|16235244|Genes, chromosome...|
|16249029|   Leukemia research|
|16266980|     Cancer research|
+--------+--------------------+
only showing top 20 rows

